# Herbert - Polish RoBERT'a model 


## Prepare the data.

* Wikipedia, Link: 
* Oscar
* Books, Polish "Wolne lektury"



Get raw text from different sources and concat in one big data file. 

Usefull shell commands

Cat all text files and instert new line between each text

```sh
find *.txt | xargs -I{} sh -c "cat '{}'; echo ''" > all_contents.txt

```

Take 11768022 first lines form splited wikipedia file
```
head -11768022 corpus_wiki_books_non_word_tokenized_2020-02-13.txt > corpus_wiki_lines_non_tok_2020-02-13.txt
```


In [1]:
import csv
import sys
import datetime as dt
import os
from pathlib import Path
import re
import tqdm as tq
from tqdm.notebook import tqdm
import mmap

print("import")

import


In [2]:
# utils functions


def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

Read wikipedia csv file and save in txt

In [4]:

wikipedia_path='./data/wikipedia_2020-02-01.csv'
output_path='./data/corpus_wikipedia_2020-02-01.txt'

csv.field_size_limit(sys.maxsize)

t0=dt.datetime.now()
with open(output_path, 'w+') as output_file:
    with open(wikipedia_path) as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for article in csv_reader:
            text = article['CleanText']

            output_file.write(text)
            output_file.write('\n')   
t1=dt.datetime.now()  
print(f'Done. Takes={t1-t0}')   

Read book dataset and normalize line splitiing. The textfile has '\n' in middle of the sentence. It is not necessary if you have proper file.

Input: concatenated book textfile
Output: file with removed new lines in the middle of the sentence. File has 


In [9]:


input_path='./data/corpus_books_2020_02_13.txt'

p = Path(input_path)
print(p.with_suffix(''))
output_path = f"{p.with_suffix('')}_norm.txt"

print(output_path)

# reg = re.compile('(?<!\.)(\n)(?=[a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ])',re.MULTILINE )
# reg = re.compile('(?<!\.)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )
#reg = re.compile('(?<!\.\n)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )

reg = re.compile('(?<=[A-Za-ząćęłńóśźż,—-])(?<!\.)(\n)(?=[a-ząćęłńóśźż])',re.MULTILINE )

# https://pymotw.com/3/mmap/#regular-expressions

line_buff = 101# 10007 # this is prime number, next is  10009  10037 
N = 0 

rep_str=' '
t0=dt.datetime.now()
with open(output_path, 'w+') as output_file:
    with open(input_path) as f:

        text=''
        for line in tqdm(f,total=get_num_lines(input_path)):
            # get block of file (line_buff) lines and replace 
            if N<line_buff:
                # glue lines
                text+=line
                N+=1
            else:
                # proces and write
                replace_text = reg.sub(rep_str, text)
                output_file.write(replace_text)
                text=''
                N=0
                
        # for the rest of the file proces and write
        if N>0:
            replace_text = reg.sub(rep_str, text)
            output_file.write(text)
            

t1=dt.datetime.now()  
print(f'Done. Takes={t1-t0}')  

data/corpus_books_2020_02_13
data/corpus_books_2020_02_13_norm.txt



Done. Takes=0:05:01.928267


## Create vocabulary


Create files for training vocabulary. One sentence per line, use polish sentence tokenizer. 
Sentence piece model is capable of handling around 12_000_000 sentences, so more sentences is not necessary.


Vocab train sets:

* Take 5M sentence from wiki, 7M from book corpus



In [11]:
import nltk

nltk.download()
nltk.download('punkt')




showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package punkt to /home/ksopyla/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:


extra_abbreviations = ['ps',  'inc', 'Corp', 'Ltd', 'Co', 'pkt',  'Dz.Ap', 'Jr', 'jr' 'sp', 'Sp', 'poj',  'pseud', 'krypt', 'sygn', 'Dz.U', 'ws', 'itd', 'np', 'sanskryt', 'nr', 'gł', 'Takht', 'tzw', 't.zw', 'ewan', 'tyt', 'oryg' ]

position_abbrev = ['Ks', 'Abp', 'abp','bp','dr', 'prof', 'zwycz', 'hab', 'arch', 'arch.kraj' 'B.Sc', 'Ph.D', 'lek', 'med', 'n.med' 'bł', 'św', 'hr' ]

roman_abbrev= [] #['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XII','XIV','XV','XVI', 'XVII', 'XVIII','XIX', 'XX', 'XXI' ]

quantity_abbrev = [ 'mln', 'tys', 'obr./min' 'godz', 'egz'] #[ 'mln', 'tys', 'km/godz', 'obr./min' 'godz', 'egz']

actions_abbrev = ['tłum','zob','wym', 'pot', 'ww', 'ogł', 'tzn', 'wyd', 'min', 'm.i', 'm.in', 'm. in' 'im','muz','tj', 'dot', 'wsp', 'właść', 'przedr', 'czyt', 'proj', 'dosł', 'hist' ]

place_abbrev = ['Śl', 'płd']

lang_abbrev = ['jęz','fr', 'ang', 'gr', 'hebr', 'czes', 'pol', 'niem', 'arab', 'egip', 'hiszp', 'jap', 'chin', 'kor', 'tyb', 'wiet']

military_abbrev = ['kpt', 'kpr', 'obs', 'pil', 'mjr','płk', 'dypl', 'pp', 'gw', 'dyw', 'bryg', 'ppłk', 'mar', 'marsz' 'rez', 'ppor', 'DPanc', 'BPanc', 'DKaw']

extra_abbreviations= extra_abbreviations + position_abbrev + roman_abbrev + quantity_abbrev + place_abbrev + actions_abbrev + place_abbrev + lang_abbrev+military_abbrev

sentence_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')
#sentence_tokenizer._params.abbrev_types.update(extra_abbreviations)

#sentences = sentence_tokenizer.tokenize(line)

files_to_proces = [
    './data/corpus_wikipedia_2020-02-01.txt',
    './data/corpus_books_2020_02_13.txt'
]
files_to_proces = [
    './data/corpus_wikipedia_2020-02-01.txt',
]
# files_to_proces = [
#     './data/corpus_books_2020_02_13.txt'
# ]

for input_file in files_to_proces:
    print(input_file)
    t0=dt.datetime.now()

    p = Path(input_file)
    output_path = f"{p.with_suffix('')}_lines.txt"

    with open(output_path, 'w+') as output_file:
        with open(input_file) as f:
            for line in tqdm(f,total=get_num_lines(input_file)):

                if line.strip() == '':
                    output_file.write('\n')
                    continue;

                sentences = sentence_tokenizer.tokenize(line)
                
                text = ''
                for sentence in sentences:
                    text += sentence.strip()
                    text+='\n'
                output_file.write(text)
                
    t1=dt.datetime.now()  
    print(f'Done. Takes={t1-t0}')   


./data/corpus_wikipedia_2020-02-01.txt


## Train the BPE sentencepiece model

We used the [sentencepiece](https://github.com/google/sentencepiece)


In [1]:
import sentencepiece as spm
import datetime

In [ ]:
max_vocab_size = 32000
model_type = "bpe"  
iss = 12_000_000

lower_vocab = False
txt_lower = "lower" if lower_vocab else "upper"
tok_model = f"wikibooks_{txt_lower}_voc_{max_vocab_size}_sent{iss}"

data_file = ''

tok_model = os.path.abspath(f"./data/{tok_model}")

piece_options = ' --bos_id=0 --eos_id=1 --pad_id=0 --unk_id=1 --shuffle_input_sentence=true'


cmd = f"--input={data_file} --model_prefix={tok_model} --model_type={model_type} --num_threads=4 --vocab_size={vocab_size}  --input_sentence_size={iss}"+ piece_options
print(cmd)


